In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("edenbd/150k-lyrics-labeled-with-spotify-valence")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/150k-lyrics-labeled-with-spotify-valence


In [4]:
!ls /kaggle/input/150k-lyrics-labeled-with-spotify-valence

labeled_lyrics_cleaned.csv


In [5]:
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader

In [6]:
ds = pd.read_csv(f"{path}/labeled_lyrics_cleaned.csv")
ds.head()

,Unnamed: 0,artist,seq,song,label
0,0,Elijah Blake,"No, no\r\nI ain't ever trapped out the bando\r...",Everyday,0.626
1,1,Elijah Blake,"The drinks go down and smoke goes up, I feel m...",Live Till We Die,0.630
2,2,Elijah Blake,She don't live on planet Earth no more\r\nShe ...,The Otherside,0.240
3,3,Elijah Blake,"Trippin' off that Grigio, mobbin', lights low\...",Pinot,0.536
4,4,Elijah Blake,"I see a midnight panther, so gallant and so br...",Shadows & Diamonds,0.371


In [7]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158353 entries, 0 to 158352
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  158353 non-null  int64  
 1   artist      158353 non-null  object 
 2   seq         158353 non-null  object 
 3   song        158351 non-null  object 
 4   label       158353 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 6.0+ MB


In [8]:
bins = [0, 0.25, 0.5, 0.75, 1.0]
ds['emotion_label'] = np.digitize(ds['label'], bins, right=False) - 1

In [9]:
ds['song_length'] = ds['seq'].apply(lambda x: len(x.split()))

In [10]:
ds.head()

,Unnamed: 0,artist,seq,song,label,emotion_label,song_length
0,0,Elijah Blake,"No, no\r\nI ain't ever trapped out the bando\r...",Everyday,0.626,2,341
1,1,Elijah Blake,"The drinks go down and smoke goes up, I feel m...",Live Till We Die,0.630,2,465
2,2,Elijah Blake,She don't live on planet Earth no more\r\nShe ...,The Otherside,0.240,0,279
3,3,Elijah Blake,"Trippin' off that Grigio, mobbin', lights low\...",Pinot,0.536,2,334
4,4,Elijah Blake,"I see a midnight panther, so gallant and so br...",Shadows & Diamonds,0.371,1,172


In [11]:
ds.describe()

,Unnamed: 0,label,emotion_label,song_length
count,158353.000000,158353.000000,158353.000000,158353.000000
mean,79176.000000,0.491052,1.459878,219.578688
std,45712.717926,0.249619,1.020068,133.818463
min,0.000000,0.000000,0.000000,0.000000
25%,39588.000000,0.286000,1.000000,135.000000
50%,79176.000000,0.483000,1.000000,190.000000
75%,118764.000000,0.691000,2.000000,263.000000
max,158352.000000,0.998000,3.000000,2843.000000


In [12]:
threshold = ds['song_length'].quantile(0.95)
threshold

np.float64(484.0)

In [13]:
from transformers import AutoTokenizer, BertForTokenClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")

2026-01-14 05:43:02.922841: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768369383.079033      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768369383.125403      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768369383.491999      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768369383.492033      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768369383.492036      55 computation_placer.cc:177] computation placer alr

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [14]:
def preprocessing(ds):
  token = tokenizer(ds['seq'], return_tensors="pt", padding='max_length', truncation='longest_first', max_length=516)
  label = torch.tensor(ds['emotion_label'])
  token['labels'] = label
  return token

In [15]:
id2label = {0: 'depressed', 1: 'sad', 2: 'neutral', 3: 'happy'}
label2id = {'depressed': 0, 'sad': 1, 'neutral': 2, 'happy': 3}

In [16]:
from transformers import AutoTokenizer, BertForSequenceClassification
import torch

model = BertForSequenceClassification.from_pretrained("google-bert/bert-base-uncased", num_labels=len(label2id), id2label=id2label, label2id=label2id)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
class LyricsDataset(Dataset):
  def __init__(self, ds, tokenizer):
    self.tokenizer = tokenizer
    self.ds = ds
  def __len__(self):
    return len(self.ds)
  def __getitem__(self, idx):
    text = self.ds.iloc[idx]['seq']
    label = self.ds.iloc[idx]['emotion_label']

    encoding = self.tokenizer(text, return_tensors="pt", padding='max_length', truncation='longest_first', max_length=512)

    # Squeeze the batch dimension added by return_tensors="pt" for a single sample
    item = {key: val.squeeze(0) for key, val in encoding.items()}
    item['labels'] = torch.tensor(label)

    return item

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Split the dataset into training and validation sets
train_ds, val_ds = train_test_split(ds, test_size=0.2, random_state=42)

# Create instances of LyricsDataset for training and validation
train_dataset = LyricsDataset(train_ds, tokenizer)
val_dataset = LyricsDataset(val_ds, tokenizer)

Next, we'll define a `compute_metrics` function to evaluate the model's performance during training. This function will calculate accuracy, precision, recall, and F1-score.

In [19]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)  #(batch, distribution)
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [20]:
from peft import LoraConfig, get_peft_model, TaskType

peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, # Sequence Classification
    inference_mode=False,
    r=8,                        # Rank (lower = fewer parameters, try 8 or 16)
    lora_alpha=32,              # Scaling factor
    lora_dropout=0.1,           # Dropout for LoRA layers
    target_modules=["query", "value"] # Target the attention layers in BERT
)

# 2. Wrap model with LoRA
lora_model = get_peft_model(model, peft_config)

lora_model.print_trainable_parameters()

trainable params: 297,988 || all params: 109,783,304 || trainable%: 0.2714


In [23]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5, # Increased epochs to allow early stopping to kick in
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    report_to='none' # Disable integration with W&B, MLflow, etc.
)

# Correctly create small datasets by slicing the underlying DataFrames
small_train_dataset = LyricsDataset(train_ds.iloc[:25000], tokenizer)
small_val_dataset = LyricsDataset(val_ds.iloc[:5000], tokenizer)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)] # Add early stopping with patience of 3 epochs
)

In [24]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.329000,1.280803,0.364800,0.389106,0.364800,0.335353
2,1.276400,1.265226,0.370800,0.392938,0.370800,0.351546
3,1.267000,1.259805,0.380600,0.389984,0.380600,0.374272
4,1.284100,1.260379,0.383000,0.397264,0.383000,0.374586
5,1.290800,1.256450,0.384200,0.395342,0.384200,0.378808


/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=2605, training_loss=1.288630694208127, metrics={'train_runtime': 6073.5839, 'train_samples_per_second': 20.581, 'train_steps_per_second': 0.429, 'total_flos': 3.3003899904e+16, 'train_loss': 1.288630694208127, 'epoch': 5.0})

In [26]:
pip install vaderSentiment

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.2 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [28]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd

analyzer = SentimentIntensityAnalyzer()

results_df = val_ds.copy()

# 1. Get Pre-trained Benchmark (VADER)
# .polarity_scores returns a 'compound' score from -1 to 1
results_df['vader_score'] = results_df['seq'].apply(lambda x: analyzer.polarity_scores(x)['compound'])

# 2. Get Your Model's Predictions (Logits/Scores)
# (Assuming you run your trainer.predict() and get the raw scores)
predictions = trainer.predict(val_dataset)
results_df['my_model_score'] = predictions.predictions.argmax(axis=1) # Or use raw logits for better correlation

/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


In [29]:
# Select the columns to compare
comparison_cols = ['label', 'vader_score', 'my_model_score']
corr_matrix = results_df[comparison_cols].corr(method='spearman') # Spearman is better for non-linear relationships

print(corr_matrix)

                   label  vader_score  my_model_score
label           1.000000     0.070019        0.405397
vader_score     0.070019     1.000000        0.132191
my_model_score  0.405397     0.132191        1.000000


In [35]:
bins = [0, 0.25, 0.5, 0.75, 1.0]
ds['emotion_label'] = np.digitize(ds['label'], bins, right=False) - 1

In [36]:
dissonant_songs = results_df[
    (results_df['emotion_label'] == 3) & 
    (results_df['my_model_score'] == 0) &
    (results_df['vader_score'] < -0.5)
]

# Display the top 5 examples
print(f"Found {len(dissonant_songs)} dissonant examples out of 5000 validation samples.")
dissonant_songs[['artist', 'song','seq', 'label', 'vader_score', 'my_model_score']].head()

Found 53 dissonant examples out of 5000 validation samples.


,artist,song,seq,label,vader_score,my_model_score
5266,Buzzcocks,Pariah,No one seems to matter anymore\r\nCan't get ba...,0.752,-0.5746,0
54233,Bad Religion,Eat Your Dog,"Weak and sick, dying in the sand, no such thin...",0.777,-0.9859,0
28826,Dead Can Dance,The Cardinal Sin,Sail to the stars on your shining desires.\r\n...,0.761,-0.8630,0
13222,Prince,The Question of U,So what is the answer to the question of you\r...,0.809,-0.6072,0
90985,The The,Giant,The sun is high and I'm surrounded by sand\r\n...,0.962,-0.9834,0


In [38]:
print(dissonant_songs.iloc[1]['seq'])
print(f"Label: {dissonant_songs.iloc[1]['emotion_label']}")
print(f"Prediction: {dissonant_songs.iloc[1]['my_model_score']}")
print(f"Vader Check: {dissonant_songs.iloc[1]['vader_score']}")


Weak and sick, dying in the sand, no such thing as a promised land.
Don't lose faith in a better life--reincarnation, poor excuse.
You're dying you assholes, your religion can't help you now.
Dying and starving in the fields you used to plow.
Rotting bones in your barren fields. 
Worshiped creature's supposed to heal.
He won't save you and he won't save me. See what you want to see.
Hindu religion in the mind of a working Joe,
Starving and dying in the fields you used to know.
You're tied and bound to a god's useless advice.
Bloated stomachs from aching diseases hold back the fight.
In the end you'll return once more to die again.
Go on 'til you can't no more in non-eternal sin.
Label: 3
Prediction: 0
Vader Check: -0.9859
